# Clustering série temporelles

## Exploration des données

In [ ]:
#conda install -c conda-forge tslearn

In [ ]:
# packages nécessaires
import os
import math

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from scipy.cluster.hierarchy import dendrogram

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.metrics import rand_score
from sklearn.metrics import adjusted_rand_score


from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from tslearn.clustering import silhouette_score as silhouette_score2
from tslearn.utils import to_time_series_dataset
from tslearn.utils import to_sklearn_dataset
from tslearn.metrics import dtw

from statsmodels.tsa.stattools import acf, pacf

In [ ]:
pip install pyarrow

In [ ]:
#base de données
df = pd.read_parquet("C:/Users/gabri/OneDrive/Bureau/Cours ENSAE/2ème année/Stage/Données/forecasting.parquet.gzip")
df

In [ ]:
df['time_index']=pd.to_datetime(df['time_index'])

On liste les séries temporelles selon les indices des produits. On effectue 3 copies : la première servira pour la distance euclidienne et les séries seront complétées par des valeurs nulles et redimensionnées, la deuxième sera redimensionnée et sera utilisé pour le clustering avec la DTW, la troisième servira seulement à effectuer des graphiques

In [ ]:
indice_produit=df['product_index'].unique()
myseries=[df[['time_index','ordered_volumes']][df['product_index']==k] for k in indice_produit]


In [ ]:
for k in range(len(myseries)):
    myseries[k].set_index('time_index',inplace=True)
    myseries[k].sort_index(inplace=True)
myseries2=myseries.copy()
myseries3=myseries.copy()
indice_temps=[series.index for series in myseries2 ]


In [ ]:
series_length=[len(series) for series in myseries]

In [ ]:
# Complétion des séries
max_len = max(series_length)
longest_series = None
for series in myseries:
    if len(series) == max_len:
        longest_series = series
for i in range(len(myseries)):
    if len(myseries[i])!= max_len:
        myseries[i] = myseries[i].reindex(longest_series.index,fill_value=0)

On redimensionne les séries temporelles

In [ ]:
for i in range(len(myseries)):
    scaler = MinMaxScaler()
    myseries[i] = MinMaxScaler().fit_transform(myseries[i])
    myseries[i]= myseries[i].reshape(len(myseries[i]))
    myseries2[i] = MinMaxScaler().fit_transform(myseries2[i])
    myseries2[i]= myseries2[i].reshape(len(myseries2[i]))

In [ ]:
X=to_time_series_dataset(myseries2)

### Clustering Kmean PCA

On effectue une PCA et on utilise la règle du coude pour savoir le nombre de composantes que l'on va choisir

In [ ]:
pca = PCA(n_components=20)
pca.fit(myseries)
plt.hist(range(1,21),weights=pca.explained_variance_ratio_,bins=20)
plt.axhline(y=0.05,color='orange')

In [ ]:
plt.plot(range(1,21),pca.explained_variance_ratio_)

In [ ]:
pca.explained_variance_ratio_

On va essayer des pca à 2 et 3 composantes

In [ ]:
pca = PCA(n_components=2)

myseries_transformed = pca.fit_transform(myseries)

In [ ]:
pca = PCA(n_components=3)

myseries_transformed3 = pca.fit_transform(myseries)

In [ ]:
myseries_transformed3

On peut traçer l'inertie pour déterminer le nombre de clusters à choisir

In [ ]:
inertie=np.empty(25,dtype='float')

for i in range(2,27):
    km = KMeans(n_clusters=i,max_iter=5000).fit(myseries_transformed3)
    inertie[i-2] = km.inertia_
plt.plot(range(2,27),inertie)

On essaye avec 3 clusters

In [ ]:
kmeans_pca = KMeans(n_clusters=3,max_iter=5000)

labels_pca = kmeans_pca.fit_predict(myseries_transformed3)

On peut traçer les clusters formés avec un kmeans sur une pca à 3 composantes: la ligne rouge représente le barycentre

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(25,5))
fig.suptitle('Clusters')
row_i=0
column_j=0
for label in set(labels_pca):
    cluster = []
    for i in range(len(labels_pca)):
            if(labels_pca[i]==label):
                axs[ column_j].plot(longest_series.index,myseries[i],c="gray",alpha=0.4)
                cluster.append(myseries[i])
    if len(cluster) > 0:
        axs[ column_j].plot(longest_series.index,np.average(np.vstack(cluster),axis=0),c="red")
    axs[column_j].set_title("Cluster "+str(row_i*4+column_j))
    column_j+=1
    if column_j%4 == 0:
        row_i+=1
        column_j=0
        

plt.savefig('C:/Users/gabri/OneDrive/Bureau/Cours ENSAE/2ème année/Stage/Résultats PCA.jpeg')
plt.show()

On peut visualiser avec la PCA 2 composantes les clusters créés

In [ ]:
cmap = plt.cm.get_cmap("tab20", kmeans_pca.cluster_centers_.shape[0])
plt.figure(figsize=(25,10))
colors_pca=[cmap(i) for i in labels_pca]
plt.scatter(myseries_transformed[:, 0], myseries_transformed[:, 1], c=colors_pca, s=300)
plt.show()

In [ ]:
fancy_names_for_labels_pca = [f"Cluster {label}" for label in labels_pca]
prédiction_pca=pd.DataFrame(zip(indice_produit,fancy_names_for_labels_pca),columns=["Series","Cluster"]).sort_values(by="Cluster").set_index("Series")

In [ ]:
prédiction_pca

In [ ]:
for k in range(3):
    print(prédiction_pca[prédiction_pca['Cluster']==f'Cluster {k}'].index)

### Kmeans

On effectue le même protocole pour les séries complètes

In [ ]:
inertie=np.empty(25,dtype='float')

for i in range(2,27):
    kmeans = KMeans(n_clusters=i,max_iter=5000)
    inertie[i-2] = kmeans.fit(myseries).inertia_
plt.plot(range(2,27),inertie)

In [ ]:
kmeans = KMeans(n_clusters=3,max_iter=5000)

labels_kmeans = kmeans.fit_predict(myseries)

In [ ]:

fig, axs = plt.subplots(1,3,figsize=(25,5))
fig.suptitle('Clusters')
row_i=0
column_j=0
for label in set(labels_kmeans):
    cluster = []
    for i in range(len(labels_kmeans)):
            if(labels_kmeans[i]==label):
                axs[ column_j].plot(longest_series.index,myseries[i],c="gray",alpha=0.4)
                cluster.append(myseries[i])
    if len(cluster) > 0:
        axs[ column_j].plot(longest_series.index,np.average(np.vstack(cluster),axis=0),c="red")
    axs[ column_j].set_title("Cluster "+str(row_i*4+column_j))
    column_j+=1
    if column_j%4 == 0:
        row_i+=1
        column_j=0
        

plt.savefig('C:/Users/gabri/OneDrive/Bureau/Cours ENSAE/2ème année/Stage/Résultat kmeans.jpeg')
plt.show()

In [ ]:
plt.figure(figsize=(25,10))
cmap = plt.cm.get_cmap("tab20", kmeans.cluster_centers_.shape[0])
colors_kmeans=[cmap(i) for i in labels_kmeans]
plt.scatter(myseries_transformed[:, 0], myseries_transformed[:, 1], c=colors_kmeans, s=300)
plt.show()

In [ ]:
fancy_names_for_labels_kmeans = [f"Cluster {label}" for label in labels_kmeans]
prédiction_kmeans=pd.DataFrame(zip(indice_produit,fancy_names_for_labels_kmeans),columns=["Series","Cluster"]).sort_values(by="Cluster").set_index("Series")

In [ ]:
prédiction_kmeans

In [ ]:
for k in range(3):
    print(prédiction_kmeans[prédiction_kmeans['Cluster']==f'Cluster {k}'].index)

### DTW

On effectue la même chose en changeant de métrique.
/!\ Le tracé de l'inertie peut prendre 4 minutes

In [ ]:
%%time
inertie=np.empty((1,10),dtype='float')

for j in range(1):
    for i in range(1,11):
        km = TimeSeriesKMeans(n_clusters=i, metric="dtw")
        inertie[j,i-1] = km.fit(X).inertia_
    print(j)
plt.plot(range(1,11),np.mean(inertie,axis=0))

In [ ]:
%%time
dtw_km = TimeSeriesKMeans(n_clusters=3, metric="dtw")

labels_dtw = dtw_km.fit_predict(X)

In [ ]:
fancy_names_for_labels_dtw = [f"Cluster {label}" for label in labels_dtw]
prediction_dtw=pd.DataFrame(zip(indice_produit,fancy_names_for_labels_dtw),columns=["Series","Cluster"]).sort_values(by="Cluster").set_index("Series")

Le barycentre est moins cohérent à tracer ici donc on trace chaque séries de chaque cluster.

/!\ Si un cluster est trop petit ($\leq$4) il faut enlever le row_i dans axs[]

In [ ]:
fig, axs = plt.subplots(len(prediction_dtw[prediction_dtw['Cluster']==f'Cluster {0}'].index)//4+1,4,figsize=(25,25))
row_i=0
column_j=0
for i in prediction_dtw[prediction_dtw['Cluster']==f'Cluster {0}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show


In [ ]:
fig, axs = plt.subplots(len(prediction_dtw[prediction_dtw['Cluster']==f'Cluster {1}'].index)//4+1,4,figsize=(25,25))
row_i=0
column_j=0
for i in prediction_dtw[prediction_dtw['Cluster']==f'Cluster {1}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show()

In [ ]:
fig, axs = plt.subplots(len(prediction_dtw[prediction_dtw['Cluster']==f'Cluster {2}'].index)//4+1,4,figsize=(25,25))
row_i=0
column_j=0
for i in prediction_dtw[prediction_dtw['Cluster']==f'Cluster {2}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
      
plt.show()

In [ ]:
for k in range(3):
    print(prediction_dtw[prediction_dtw['Cluster']==f'Cluster {k}'].index)

### Hierarchical Clustering

On utilise un autre algorithme qui est le Hierarchical clustering avec toujours la DTW comme métrique. On va essayer les 3 types de linkage

In [ ]:
dtw_precomputed=np.zeros((125,125))
for i in range(125):
    for j in range(125):
        dtw_precomputed[i,j]=dtw(myseries2[i], myseries2[j])

In [ ]:
hac_average=AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='average',compute_distances=True)
hac_single=AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='single',compute_distances=True)
hac_complete=AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='complete',compute_distances=True)
modele_average = hac_average.fit(dtw_precomputed)
modele_single = hac_single.fit(dtw_precomputed)
modele_complete = hac_complete.fit(dtw_precomputed)

On affiche les dendrogrammes des 3 types de linkage

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
plot_dendrogram(modele_average, truncate_mode="level", p=5)

In [ ]:
plot_dendrogram(modele_single, truncate_mode="level", p=10)

In [ ]:
plot_dendrogram(modele_complete, truncate_mode="level", p=4)

Le linkage single est pas très intéréssant mais les deux autres le sont et surtout le complete qui sépare bien les clusters que l'on va utiliser pour afficher les clusters.

In [ ]:
hac=AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='complete')

labels_hac = hac.fit_predict(dtw_precomputed)

In [ ]:
fancy_names_for_labels_hac = [f"Cluster {label}" for label in labels_hac]
prediction_hac=pd.DataFrame(zip(indice_produit,fancy_names_for_labels_hac),columns=["Series","Cluster"]).sort_values(by="Cluster").set_index("Series")

In [ ]:
fig, axs = plt.subplots(len(prediction_hac[prediction_hac['Cluster']==f'Cluster {0}'].index)//4+1,4,figsize=(25,35))
row_i=0
column_j=0
for i in prediction_hac[prediction_hac['Cluster']==f'Cluster {0}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
     
plt.savefig('C:/Users/gabri/OneDrive/Bureau/Cours ENSAE/2ème année/Stage/Cluster 1 hierarchical clustering.jpeg')
plt.show()

In [ ]:
fig, axs = plt.subplots(len(prediction_hac[prediction_hac['Cluster']==f'Cluster {1}'].index)//4+1,4,figsize=(25,10))
row_i=0
column_j=0
for i in prediction_hac[prediction_hac['Cluster']==f'Cluster {1}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0

plt.savefig('C:/Users/gabri/OneDrive/Bureau/Cours ENSAE/2ème année/Stage/Cluster 2 hierarchical clustering.jpeg')
plt.show()

In [ ]:
fig, axs = plt.subplots(len(prediction_hac[prediction_hac['Cluster']==f'Cluster {2}'].index)//4+1,4,figsize=(25,25))
row_i=0
column_j=0
for i in prediction_hac[prediction_hac['Cluster']==f'Cluster {2}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
        
plt.savefig('C:/Users/gabri/OneDrive/Bureau/Cours ENSAE/2ème année/Stage/Cluster 3 hierarchical clustering.jpeg')    
plt.show()

In [ ]:
for k in range(3):
    print(prediction_hac[prediction_hac['Cluster']==f'Cluster {k}'].index)

### DBSCAN

On va utiliser l'algorithme de DBSCAN. On prend e tel qu'une part « suffisamment grande » des points aient une distance à son plus proche voisin inférieure à e et n tel qu'une part « suffisamment grande » des points aient plus de n points dans leur e-voisinage.

In [ ]:
# Recherche des paramètres
a=np.copy(dtw_precomputed)
e1=np.quantile(np.sort(a,axis=0)[1],0.95)
n1=np.quantile(np.count_nonzero(a<=e1,axis=0),0.05)
e2=np.quantile(np.sort(a,axis=0)[1],0.99)
n2=np.quantile(np.count_nonzero(a<=e2,axis=0),0.01)

In [ ]:
%%time
dbscan=DBSCAN(eps=e1,min_samples=n1,metric='precomputed')

labels_dbscan = dbscan.fit_predict(dtw_precomputed)

In [ ]:
fancy_names_for_labels_dbscan= [f"Cluster {label}" for label in labels_dbscan]
prediction_dbscan=pd.DataFrame(zip(indice_produit,fancy_names_for_labels_dbscan),columns=["Series","Cluster"]).sort_values(by="Cluster").set_index("Series")

In [ ]:
prediction_dbscan['Cluster'].unique().shape

In [ ]:
prediction_dbscan

In [ ]:
for k in range(-1,2):
    print(prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {k}'].index)

In [ ]:
fig, axs = plt.subplots(len(prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {-1}'].index)//4+1,4,figsize=(25,50))
row_i=0
column_j=0
for i in prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {-1}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show
plt.savefig('C:/Users/gabri/OneDrive/Bureau/Cours ENSAE/2ème année/Stage/Cluster autres DBSCAN.jpeg')

In [ ]:
fig, axs = plt.subplots(len(prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {0}'].index)//4+1,4,figsize=(25,50))
row_i=0
column_j=0
for i in prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {0}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show
plt.savefig('C:/Users/gabri/OneDrive/Bureau/Cours ENSAE/2ème année/Stage/Cluster 1 DBSCAN.jpeg')

In [ ]:
fig, axs = plt.subplots(len(prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {1}'].index)//4+1,4,figsize=(25,5))
row_i=0
column_j=0
for i in prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {1}'].index:
    axs[column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show
plt.savefig('C:/Users/gabri/OneDrive/Bureau/Cours ENSAE/2ème année/Stage/Cluster 2 DBSCAN.jpeg')

### Comparaison Kmeans, PCA et DTW

On va comparer l'algorithme de kmeans avec l'inertie. Le premier tableau est la moyenne et le deuxième la variance

Ca peut être un peu long selon le nombre d'itération à faire (environ 1 minute 20 par itération)

In [ ]:
%%time
modeles_inertie=np.zeros((1,3,4))
for k in range(1):
    modeles_inertie[k]=np.array([[KMeans(n_clusters=3,max_iter=5000).fit(myseries_transformed).inertia_,KMeans(n_clusters=4,max_iter=5000).fit(myseries_transformed3).inertia_,KMeans(n_clusters=5,max_iter=5000).fit(myseries_transformed).inertia_,KMeans(n_clusters=6,max_iter=5000).fit(myseries_transformed).inertia_],
                         [KMeans(n_clusters=3,max_iter=5000).fit(myseries).inertia_,KMeans(n_clusters=4,max_iter=5000).fit(myseries).inertia_,KMeans(n_clusters=5,max_iter=5000).fit(myseries).inertia_,KMeans(n_clusters=6,max_iter=5000).fit(myseries).inertia_],
                          [TimeSeriesKMeans(n_clusters=3, metric="dtw").fit(X).inertia_,TimeSeriesKMeans(n_clusters=4, metric="dtw").fit(X).inertia_,TimeSeriesKMeans(n_clusters=5, metric="dtw").fit(X).inertia_,TimeSeriesKMeans(n_clusters=6, metric="dtw").fit(X).inertia_]])
    print(k)


In [ ]:
fig, ax =plt.subplots(1,1,figsize=(10,10))

column_labels=["n=3", "n=4", "n=5","n=6"]
row_labels=["PCA 3 Composantes","Kmeans","DTW"]
ax.axis('tight')
ax.axis('off')
ax.table(cellText=np.mean(modeles_inertie, axis=0),colLabels=column_labels,rowLabels=row_labels,loc="center")

plt.show()

In [ ]:
fig, ax =plt.subplots(1,1,figsize=(10,10))

column_labels=["n=3", "n=4", "n=5","n=6"]
row_labels=["PCA 3 Composantes","Kmeans","DTW"]
ax.axis('tight')
ax.axis('off')
ax.table(cellText=np.var(modeles_inertie, axis=0),colLabels=column_labels,rowLabels=row_labels,loc="center")

plt.show()

La DTW est beaucoup plus performante, on va donc la comparer aux autres algorithmes avec différentes mesures

### Comparaison avec d'autres mesures de score

#### Silhouette score

On peut d'abord regarder le score silhouette 

Ca prend environ 3 minutes par itération

In [ ]:
%%time
#modeles_silhouette=np.zeros((1,5,4))
#a=silhouette_score2(X,DBSCAN(eps=e1,min_samples=n1,metric='precomputed').fit_predict(dtw_precomputed))
#b=silhouette_score2(X,DBSCAN(eps=e2,min_samples=n2,metric='precomputed').fit_predict(dtw_precomputed))
#c=np.array([silhouette_score2(X,AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),silhouette_score2(X,AgglomerativeClustering(n_clusters=4,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),silhouette_score2(X,AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),silhouette_score2(X,AgglomerativeClustering(n_clusters=6,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed))])
#d= np.array([silhouette_score2(X,AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),silhouette_score2(X,AgglomerativeClustering(n_clusters=4,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),silhouette_score2(X,AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),silhouette_score2(X,AgglomerativeClustering(n_clusters=6,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed))])
#for k in range(1):
#    modeles_silhouette[k,0]=np.array([silhouette_score2(X,TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X)),silhouette_score2(X,TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X)),silhouette_score2(X,TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X)),silhouette_score2(X,TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X))])
#    modeles_silhouette[k,1]=c
#    modeles_silhouette[k,2]=d                                
#    modeles_silhouette[k,3]=np.full(4,a)
#    modeles_silhouette[k,4]=np.full(4,b)
#    print(k)


In [ ]:
#fig, ax =plt.subplots(1,1,figsize=(10,10))

#column_labels=["n=3", "n=4", "n=5","n=6"]
#row_labels=["DTW","HAC Complete","HAC average","DBSCAN eps=1,82 min_samples=2","DBSCAN eps=1,72 min_samples=2"]
#ax.axis('tight')
#ax.axis('off')
#ax.table(cellText=np.mean(modeles_silhouette, axis=0),colLabels=column_labels,rowLabels=row_labels,loc="center")

#plt.show()

In [ ]:
#fig, ax =plt.subplots(1,1,figsize=(10,10))

#column_labels=["n=3", "n=4", "n=5","n=6"]
#row_labels=["DTW","HAC Complete","HAC average","DBSCAN eps=1,82 min_samples=2","DBSCAN eps=1,72 min_samples=2"]
#ax.axis('tight')
#ax.axis('off')
#ax.table(cellText=np.var(modeles_silhouette, axis=0),colLabels=column_labels,rowLabels=row_labels,loc="center")

#plt.show()

#### Score de Calinski-Harabasz 

On regarde ensuite le score de Calinski-Harabasz.

Ca prend 6 minutes par itération

In [ ]:
def BGSS(X,labels):
    bgss=0
    C=dtw_barycenter_averaging(X)
    for k in np.unique(labels):
        n_k=len(np.where(labels==k)[0])
        C_k=dtw_barycenter_averaging(X[np.where(labels==k)])
        bgss+=n_k*dtw(C_k,C)**2
    return bgss

In [ ]:
def WGSS(X,labels):
    WGSS=0
    for k in np.unique(labels):
        WGSSk=0
        C_k=dtw_barycenter_averaging(X[np.where(labels==k)])
        for x in np.where(labels==k)[0]:
            WGSSk+=dtw(C_k,X[x])**2
        WGSS+=WGSSk
    return WGSS

In [ ]:
def Calinski_Harabasz_score(X,labels):
    N=len(X)
    K=len(np.unique(labels))
    return BGSS(X,labels)*(N-K)/(WGSS(X,labels)*(K-1))

In [ ]:
#%%time
#modeles_calinski_harabasz=np.zeros((1,5,4))
#a=Calinski_Harabasz_score(X,DBSCAN(eps=e1,min_samples=n1,metric='precomputed').fit_predict(dtw_precomputed))
#b=Calinski_Harabasz_score(X,DBSCAN(eps=e2,min_samples=n2,metric='precomputed').fit_predict(dtw_precomputed))
#c=np.array([Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=4,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=6,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed))])
#d=np.array([Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=4,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=6,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed))])
#for k in range(1):
#    modeles_calinski_harabasz[k,0]=np.array([Calinski_Harabasz_score(X,TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X)),Calinski_Harabasz_score(X,TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X)),Calinski_Harabasz_score(X,TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X)),Calinski_Harabasz_score(X,TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X))])
#    modeles_calinski_harabasz[k,1]=c
#    modeles_calinski_harabasz[k,2]=d                                  
#    modeles_calinski_harabasz[k,3]=np.full(4,a)
#    modeles_calinski_harabasz[k,4]=np.full(4,b)
#    print(k)


In [ ]:
#fig, ax =plt.subplots(1,1,figsize=(10,10))

#column_labels=["n=3", "n=4", "n=5","n=6"]
#row_labels=["DTW","HAC Complete","HAC average","DBSCAN eps=1,82 min_samples=2","DBSCAN eps=1,72 min_samples=2"]
#ax.axis('tight')
#ax.axis('off')
#ax.table(cellText=np.mean(modeles_calinski_harabasz, axis=0),colLabels=column_labels,rowLabels=row_labels,loc="center")

#plt.show()

In [ ]:
#fig, ax =plt.subplots(1,1,figsize=(10,10))

#column_labels=["n=3", "n=4", "n=5","n=6"]
#row_labels=["DTW","HAC Complete","HAC average","DBSCAN eps=1,82 min_samples=2","DBSCAN eps=1,72 min_samples=2"]
#ax.axis('tight')
#ax.axis('off')
#ax.table(cellText=np.var(modeles_calinski_harabasz, axis=0),colLabels=column_labels,rowLabels=row_labels,loc="center")

#plt.show()

#### Davies-Bouldin score

On fait de même avec le score de Davies-Bouldin.

Une itération dure environ 32 minutes 

In [ ]:
def distance_intra_cluster(X,labels,k):
    d=0
    C_k=dtw_barycenter_averaging(X[np.where(labels==k)])
    for x in np.where(labels==k)[0]:
            d+=dtw(C_k,X[x])
    return d

In [ ]:
def Davies_Bouldin_score(X,labels):
    K=len(np.unique(labels))
    score=0
    for k in np.unique(labels):
        maxi=0
        for k2 in np.unique(labels):
            if k2!=k:
                if (distance_intra_cluster(X,labels,k)+distance_intra_cluster(X,labels,k2))/dtw(dtw_barycenter_averaging(X[np.where(labels==k)]),dtw_barycenter_averaging(X[np.where(labels==k2)]))>maxi:
                    maxi=(distance_intra_cluster(X,labels,k)+distance_intra_cluster(X,labels,k2))/dtw(dtw_barycenter_averaging(X[np.where(labels==k)]),dtw_barycenter_averaging(X[np.where(labels==k2)]))
        score+=maxi
    return score

In [ ]:
#%%time
#modeles_davies_bouldin=np.zeros((1,5,4))
#a=Davies_Bouldin_score(X,DBSCAN(eps=e1,min_samples=n1,metric='precomputed').fit_predict(dtw_precomputed))
#b=Davies_Bouldin_score(X,DBSCAN(eps=e2,min_samples=n2,metric='precomputed').fit_predict(dtw_precomputed))
#c=np.array([Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=4,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=6,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed))])
#d=np.array([Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=4,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=6,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed))])
#for k in range(1):
#    modeles_davies_bouldin[k,0]=np.array([Davies_Bouldin_score(X,TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X)),Davies_Bouldin_score(X,TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X)),Davies_Bouldin_score(X,TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X)),Davies_Bouldin_score(X,TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X))])
#    modeles_davies_bouldin[k,1]=c
#    modeles_davies_bouldin[k,2]=d                                  
#    modeles_davies_bouldin[k,3]=np.full(4,a)
#    modeles_davies_bouldin[k,4]=np.full(4,b)
#    print(k)


In [ ]:
#fig, ax =plt.subplots(1,1,figsize=(10,10))

#column_labels=["n=3", "n=4", "n=5","n=6"]
#row_labels=["DTW","HAC Complete","HAC average","DBSCAN eps=1,82 min_samples=2","DBSCAN eps=1,72 min_samples=2"]
#ax.axis('tight')
#ax.axis('off')
#ax.table(cellText=np.mean(modeles_davies_bouldin, axis=0),colLabels=column_labels,rowLabels=row_labels,loc="center")

#plt.show()


In [ ]:
#fig, ax =plt.subplots(1,1,figsize=(10,10))

#column_labels=["n=3", "n=4", "n=5","n=6"]
#row_labels=["DTW","HAC Complete","HAC average","DBSCAN eps=1,82 min_samples=2","DBSCAN eps=1,72 min_samples=2"]
#ax.axis('tight')
#ax.axis('off')
#ax.table(cellText=np.var(modeles_davies_bouldin, axis=0),colLabels=column_labels,rowLabels=row_labels,loc="center")

#plt.show()

### Toujours ensemble 3 clusters

On regarde ensuite les indices qui sont toujours regroupés ensemble. On va le faire quand on prend 3 clusters puis 4 puis 5 et enfin 6

In [ ]:
def toujours_ensemble(i,j,*args):
    for k in args:
        if k[i]!=k[j]:
            return False
    return True

In [ ]:
def dans_liste_de_liste(l,i):
    for k in l:
        if i in k:
            return True
    return False

In [ ]:
def indices_toujours_ensemble(*args):
    l=[]
    for i in range(125):
        li=[]
        if not dans_liste_de_liste(l,indice_produit[i]):
            for j in range(125):
                if toujours_ensemble(i,j,*args):
                    li.append(indice_produit[j])
            if len(li)>1:
                l.append(li)
    return l

In [ ]:
c_trois=TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X)

In [ ]:
argument3=c_trois,TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)

In [ ]:
%%time

a=indices_toujours_ensemble(*argument3)

In [ ]:
len(a)

In [ ]:
sum([len(k) for k in a])

In [ ]:
[len(k) for k in a]

### Toujours ensemble 4 clusters 

In [ ]:
c_quatre=TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X)

In [ ]:
argument4=c_quatre,TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X),AgglomerativeClustering(n_clusters=4,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)

In [ ]:
b=indices_toujours_ensemble(*argument4)

In [ ]:
len(b)

In [ ]:
sum([len(k) for k in b])

In [ ]:
[len(k) for k in b]

In [ ]:
b[1]

### Toujours ensemble 5 clusters

In [ ]:
c_cinq=TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X)

In [ ]:
argument5=c_cinq,TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X),AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)

In [ ]:
c=indices_toujours_ensemble(*argument5)

In [ ]:
len(c)

In [ ]:
sum([len(k) for k in c])

In [ ]:
[len(k) for k in c]

In [ ]:
c[0]

### Toujours ensemble 6 clusters

In [ ]:
c_six=TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X)

In [ ]:
argument6=c_six,TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X),AgglomerativeClustering(n_clusters=6,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)

In [ ]:
d=indices_toujours_ensemble(*argument6)

In [ ]:
len(d)

In [ ]:
sum([len(k) for k in d])

In [ ]:
[len(k) for k in d]

Cette dernière ligne permet de traçer les séries des indices d'une certaine liste. Il faut remplacer a[1] par la liste d'indice que vous voulez.

/!\ Si liste de longueur plus petite que 4 enlever le row_i dans axs

In [ ]:
#fig, axs = plt.subplots(len(a[1])//4+1,4,figsize=(25,35))
#row_i=0
#column_j=0
#for i in a[1]:
#    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
#    column_j+=1
#    if column_j%4==0:
#        row_i+=1
#        column_j=0
     
#plt.show()